In [1]:
!git clone https://github.com/hpcaitech/ColossalAI.git
!cd ColossalAI
# install dependency
!pip install -r ColossalAI/requirements/requirements.txt
# install colossalai
!pip install ColossalAI/

Cloning into 'ColossalAI'...
remote: Enumerating objects: 1457, done.
remote: Counting objects: 100% (476/476), done.
remote: Compressing objects: 100% (248/248), done.
remote: Total 1457 (delta 285), reused 306 (delta 224), pack-reused 981
Receiving objects: 100% (1457/1457), 898.07 KiB | 6.24 MiB/s, done.
Resolving deltas: 100% (706/706), done.
Processing ./ColossalAI
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for colossalai: filename=colossalai-0.0.1b0-py3-none-any.whl size=245405 sha256=0550a185c031bd45ac100d51656dcf4759a778f463b58eeb1cb2fbdfbdfe55f9
  Stored in directory: /tmp/pip-ephem-wheel-cache-v0lu3b4q/wheels/03/f

In [2]:
!git clone https://github.com/NVIDIA/apex
!pip install -v --disable-pip-version-check ./apex

Cloning into 'apex'...
remote: Enumerating objects: 8800, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 8800 (delta 12), reused 11 (delta 0), pack-reused 8767
Receiving objects: 100% (8800/8800), 14.46 MiB | 20.63 MiB/s, done.
Resolving deltas: 100% (5993/5993), done.
Using pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Value for scheme.platlib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.purelib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.headers does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/include/python3.7/UNKNOWN
sysconfig: /usr/include/python3.7

In [3]:
import torch
import colossalai
from colossalai.engine import Engine
from colossalai.context import Config

Colossalai should be built with cuda extension to use the FP16 optimizer


In [4]:
# initialize distributed
CONFIG = Config(
    dict(
        parallel=dict(
            data=dict(size=1),
            pipeline=dict(size=1),
            tensor=dict(size=1, mode=None),
            )
        )
    )
colossalai.launch(config=CONFIG,
                  rank=0,
                  world_size=1,
                  host='127.0.0.1',
                  port=8888,
                  backend='nccl')

colossalai - torch.distributed.distributed_c10d - 2021-12-20 01:11:46,597 INFO: Added key: store_based_barrier_key:1 to store for rank: 0
colossalai - torch.distributed.distributed_c10d - 2021-12-20 01:11:46,605 INFO: Rank 0: Completed store-based barrier for key:store_based_barrier_key:1 with 1 nodes.
colossalai - torch.distributed.distributed_c10d - 2021-12-20 01:11:46,610 INFO: Added key: store_based_barrier_key:2 to store for rank: 0
colossalai - torch.distributed.distributed_c10d - 2021-12-20 01:11:46,612 INFO: Rank 0: Completed store-based barrier for key:store_based_barrier_key:2 with 1 nodes.
colossalai - torch.distributed.distributed_c10d - 2021-12-20 01:11:46,619 INFO: Added key: store_based_barrier_key:3 to store for rank: 0
colossalai - torch.distributed.distributed_c10d - 2021-12-20 01:11:46,623 INFO: Rank 0: Completed store-based barrier for key:store_based_barrier_key:3 with 1 nodes.
colossalai - root - 2021-12-20 01:11:46,629 INFO: process rank 0 is bound to device 0
co

In [5]:
# create dataloader
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets

transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,)),
            ]
        )

batch_size = 10 

trainset = torchvision.datasets.MNIST(root='./data', train = True, transform = transform, download = True)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train = False, transform = transform, download = True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [6]:
# construct the model
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.fc1 = nn.Linear(28*28, 100)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(100, 10)
  
  def forward(self,x):
    x = x.view(-1,784)
    a = self.fc1(x)
    a = self.relu(a)
    a = self.fc2(a)
    return a

model = Net().cuda()  

In [7]:
# build optimizer and loss
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.001,momentum=0.9)

In [8]:
# intiialize with colossalai
engine, trainloader, testloader, _ = colossalai.initialize(model, optimizer, criterion, trainloader, testloader)

colossalai - root - 2021-12-20 01:12:16,492 INFO: 
========== Your Config ========
{'parallel': {'data': {'size': 1},
              'pipeline': {'size': 1},
              'tensor': {'mode': None, 'size': 1}}}

colossalai - root - 2021-12-20 01:12:16,496 INFO: cuDNN benchmark = True, deterministic = False
colossalai - root - 2021-12-20 01:12:16,500 WARNING: No PyTorch DDP or gradient handler is set up, please make sure you do not need to all-reduce the gradients after a training step.


In [10]:
for epoch in range(10):
    # train
    engine.train()

    for img, label in trainloader:
        img = img.cuda()
        # you need flatten the image from (b, 1, 28, 28) to (b, 784)
        # b is batch size
        img = img.flatten(start_dim=1)
        label = label.cuda()
        engine.zero_grad()
        output = engine(img)
        loss = engine.criterion(output, label)
        engine.backward(loss)
        engine.step()

    # test
    engine.eval()
    correct = 0
    total = 0

    for img, label in trainloader:
        img = img.cuda()
        img = img.flatten(start_dim=1)
        label = label.cuda()
        with torch.no_grad():
            output = engine(img)
        prediction = torch.argmax(output, dim=-1)
        correct += torch.sum(prediction == label)
        total += img.size(0)
    
    print(f'epoch {epoch} - acc: {correct/total}')

epoch 0 - acc: 0.9831333160400391
epoch 1 - acc: 0.9853166937828064
epoch 2 - acc: 0.9859833717346191
epoch 3 - acc: 0.9904666543006897
epoch 4 - acc: 0.9914000034332275
epoch 5 - acc: 0.9928500056266785
epoch 6 - acc: 0.9940666556358337
epoch 7 - acc: 0.9950667023658752
epoch 8 - acc: 0.9958166480064392
epoch 9 - acc: 0.9971166849136353
